<a href="https://colab.research.google.com/github/BobSheehan23/Bob_EquiLend_Models/blob/main/Heavil_Shorted_YTD_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# --- 1. Load Data & Initial Setup ---
# We begin by loading the dataset. The 'business_date' column is parsed as a date object,
# which is crucial for time-series analysis. This ensures that we can correctly identify
# the start and end of the period for calculations like Year-to-Date returns.
try:
    df = pd.read_csv('R3K_SecLending_after_20241231_clean.csv', parse_dates=['business_date'])
    print("Step 1: Data loaded successfully.")
except FileNotFoundError:
    print("Error: 'R3K_SecLending_after_20241231_clean.csv' not found.")
    print("Please ensure the CSV file is in the same directory as this notebook.")
    # Exit or create a dummy dataframe to avoid further errors in the script
    df = pd.DataFrame()

if not df.empty:
    # --- 2. Calculate Value-Based Utilization ---
    # Value-Based Utilization is a key metric in securities lending that indicates how much of
    # a security's available inventory is being borrowed. It's calculated as the ratio of
    # the value of borrowed shares to the total inventory value.
    # We convert 'borrow_value' and 'inventory_val_amt' to numeric types, coercing any
    # errors into 'Not a Number' (NaN). This prevents non-numeric data from breaking our calculations.
    # Rows with missing essential data for this calculation are dropped to ensure data quality.
    # To handle cases where inventory value is zero (which would cause a division by zero error),
    # we replace the resulting 'infinity' values with NaN.
    df['borrow_value'] = pd.to_numeric(df['borrow_value'], errors='coerce')
    df['inventory_val_amt'] = pd.to_numeric(df['inventory_val_amt'], errors='coerce')
    df.dropna(subset=['borrow_value', 'inventory_val_amt'], inplace=True)
    df['utilization'] = df['borrow_value'] / df['inventory_val_amt']
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    print("Step 2: Value-Based Utilization calculated.")

    # --- 3. Apply Value-Based Gating ---
    # To focus our analysis on securities with significant lending activity, we apply a 'gating'
    # procedure. This filters the universe of stocks to include only those that meet certain
    # criteria on the most recent trading day in the dataset. This ensures our analysis is
    # focused on relevant, actively lent names.
    latest_date = df['business_date'].max()
    latest_day_data = df[df['business_date'] == latest_date]

    gating_criteria = (latest_day_data['borrow_value'] >= 1_000_000) & \
                      (latest_day_data['inventory_val_amt'] >= 10_000_000)

    gated_tickers = latest_day_data[gating_criteria]['ticker'].unique()
    gated_df = df[df['ticker'].isin(gated_tickers)].copy()
    print(f"Step 3: Gating applied. {len(gated_tickers)} stocks met the criteria.")

    # --- 4. Calculate YTD Returns & Average Utilization ---
    # For the gated stocks, we now calculate Year-to-Date (YTD) returns and average utilization.
    # YTD return is calculated as the percentage change in price from the first to the last
    # day of the period for each stock. Average utilization is the mean of the daily
    # utilization over the entire period. These two metrics will be the basis of our analysis.
    gated_df.sort_values(by=['ticker', 'business_date'], inplace=True)

    # Calculate YTD returns
    start_prices = gated_df.groupby('ticker')['security_price'].first()
    end_prices = gated_df.groupby('ticker')['security_price'].last()
    ytd_returns = ((end_prices - start_prices) / start_prices)
    ytd_returns.name = 'ytd_return'

    # Calculate average utilization
    avg_utilization = gated_df.groupby('ticker')['utilization'].mean()
    avg_utilization.name = 'avg_utilization'

    # Combine into a single analysis DataFrame
    analysis_df = pd.concat([ytd_returns, avg_utilization], axis=1)
    analysis_df.dropna(inplace=True) # Drop stocks with missing data for these metrics
    print("Step 4: YTD Returns and Average Utilization calculated.")
    print("\nSample of the analysis DataFrame:")
    print(analysis_df.head())


    # --- 5. Decile Analysis ---
    # To understand the relationship between utilization and performance, we perform a decile analysis.
    # Stocks are sorted by their average utilization and then grouped into ten equal-sized buckets (deciles).
    # We then calculate the mean YTD return for each decile. This may indicate if
    # higher utilization is associated with different return profiles.
    analysis_df['decile'] = pd.qcut(analysis_df['avg_utilization'], 10, labels=False, duplicates='drop')
    decile_analysis = analysis_df.groupby('decile')['ytd_return'].agg(['mean'])
    print("\n--- Step 5: Decile Analysis ---")
    print("Mean YTD Return by Utilization Decile:")
    print(decile_analysis)


    # --- 6. Extreme Group Analysis ---
    # This analysis focuses on the extremes. We look at the performance of the top and bottom 50
    # stocks by average utilization, as well as the top and bottom 1%. This can highlight
    # performance characteristics of the most and least crowded shorts in the market.
    # This may provide signals about market sentiment at its extremes.
    print("\n--- Step 6: Extreme Group Analysis ---")
    sorted_analysis_df = analysis_df.sort_values(by='avg_utilization', ascending=False)

    # Top and Bottom 50 Analysis
    top_50 = sorted_analysis_df.head(50)
    bottom_50 = sorted_analysis_df.tail(50)

    print("\nPerformance of Top 50 Stocks by Average Utilization:")
    print(top_50['ytd_return'].agg(['mean']))

    print("\nPerformance of Bottom 50 Stocks by Average Utilization:")
    print(bottom_50['ytd_return'].agg(['mean']))


    # Top and Bottom 1% Analysis
    # This is performed only if the gated universe is large enough to yield meaningful results.
    num_stocks = len(analysis_df)
    if num_stocks >= 100:
        one_percent_count = int(num_stocks * 0.01)
        if one_percent_count > 0:
            top_1_percent = sorted_analysis_df.head(one_percent_count)
            bottom_1_percent = sorted_analysis_df.tail(one_percent_count)

            print(f"\nPerformance of Top 1% ({one_percent_count} stocks) by Average Utilization:")
            print(top_1_percent['ytd_return'].agg(['mean']))

            print(f"\nPerformance of Bottom 1% ({one_percent_count} stocks) by Average Utilization:")
            print(bottom_1_percent['ytd_return'].agg(['mean']))
        else:
            print("\nNot enough stocks for a meaningful 1% analysis (1% is less than 1 stock).")
    else:
        print("\nSkipping 1% analysis because there are fewer than 100 stocks in the gated universe.")

# Task
Generate a market flash document titled "Market Flash: Most Shorted Stocks" that includes the provided text and two bar charts visualizing the decile analysis and extreme group analysis of stock returns based on utilization rates. The charts should replace the tables in the original analysis. Use the data from the `decile_analysis` DataFrame for the decile chart and the calculated mean YTD returns for the extreme groups (Top 50, Bottom 50, Top 1%, Bottom 1%) for the extreme group chart.

## Visualize decile analysis

### Subtask:
Create a bar chart showing the mean YTD return for each utilization decile based on the `decile_analysis` DataFrame.


**Reasoning**:
Generate a bar chart to visualize the mean YTD return for each utilization decile using the `decile_analysis` DataFrame.



In [ ]:
import matplotlib.pyplot as plt

# --- Chart Colors ---
text_color = '#374151'
title_color = '#006db7'
negative_color = '#f57600'
positive_color = '#048dd2'


plt.figure(figsize=(10, 6))

# Assign colors based on whether the mean return is negative or positive
colors = [negative_color if mean < 0 else positive_color for mean in decile_analysis['mean']]
plt.bar(decile_analysis.index, decile_analysis['mean'], color=colors)


plt.xlabel('Utilization Decile', fontsize=12, fontweight='bold', color=text_color)
plt.ylabel('Mean YTD Return', fontsize=12, fontweight='bold', color=text_color)
plt.title('Mean YTD Return by Utilization Decile', fontsize=14, fontweight='bold', color=title_color)
plt.xticks(decile_analysis.index)
plt.yticks(color=text_color, fontweight='bold')
# Remove gridlines
plt.grid(False)
plt.show()

## Visualize extreme group analysis

### Subtask:
Create a bar chart comparing the mean YTD returns of the Top 50, Bottom 50, Top 1%, and Bottom 1% groups based on the previously printed analysis results.


**Reasoning**:
Create a bar chart to visualize the mean YTD returns for the extreme utilization groups using the previously calculated values.



In [ ]:
import matplotlib.pyplot as plt

# Extract mean returns for extreme groups from previous analysis output
extreme_group_returns = {
    "Top 50": top_50['ytd_return'].mean(),
    "Bottom 50": bottom_50['ytd_return'].mean(),
    "Top 1%": top_1_percent['ytd_return'].mean(),
    "Bottom 1%": bottom_1_percent['ytd_return'].mean()
}

labels = list(extreme_group_returns.keys())
returns = list(extreme_group_returns.values())

# --- Chart Colors ---
text_color = '#374151'
title_color = '#006db7'
colors = ['#048dd2', '#f57600', '#048dd2', '#f57600'] # Using provided colors for bars

plt.figure(figsize=(8, 5))
plt.bar(labels, returns, color=colors)
plt.xlabel('Group', fontsize=12, fontweight='bold', color=text_color)
plt.ylabel('Mean YTD Return', fontsize=12, fontweight='bold', color=text_color)
plt.title('Mean YTD Return for Extreme Utilization Groups', fontsize=14, fontweight='bold', color=title_color)
plt.yticks(color=text_color, fontweight='bold')
# Remove gridlines
plt.grid(False)
plt.show()

## Combine text and charts

### Subtask:
Create a markdown cell that includes the provided market flash text and the generated charts.


**Reasoning**:
Create a markdown cell with the market flash text and the previously generated charts.



In [ ]:
%%markdown
# Market Flash: Most Shorted Stocks

This market flash analyzes the relationship between stock utilization rates (a proxy for short interest) and Year-to-Date (YTD) stock returns for a universe of stocks that meet certain value-based gating criteria as of the most recent trading day in the dataset (July 25, 2025).

The analysis focuses on two key aspects:
1.  **Decile Analysis:** Examining the mean YTD return for stocks grouped into deciles based on their average utilization rate over the period.
2.  **Extreme Group Analysis:** Comparing the mean YTD returns of the top and bottom 50 stocks, as well as the top and bottom 1% of stocks by average utilization.

## Decile Analysis: Mean YTD Return by Utilization Decile

The chart below illustrates the mean YTD return for each decile of utilization. We see a general trend where higher utilization deciles tend to exhibit higher mean YTD returns, with a significant jump in performance for the highest decile.


**Reasoning**:
Continue adding the remaining text and the extreme group analysis chart to the markdown cell.



In [ ]:
%%markdown
# Market Flash: Most Shorted Stocks

This market flash analyzes the relationship between stock utilization rates (a proxy for short interest) and Year-to-Date (YTD) stock returns for a universe of stocks that meet certain value-based gating criteria as of the most recent trading day in the dataset (July 25, 2025).

The analysis focuses on two key aspects:
1.  **Decile Analysis:** Examining the mean YTD return for stocks grouped into deciles based on their average utilization rate over the period.
2.  **Extreme Group Analysis:** Comparing the mean YTD returns of the top and bottom 50 stocks, as well as the top and bottom 1% of stocks by average utilization.

## Decile Analysis: Mean YTD Return by Utilization Decile

The chart below illustrates the mean YTD return for each decile of utilization. We see a general trend where higher utilization deciles tend to exhibit higher mean YTD returns, with a significant jump in performance for the highest decile.

## Extreme Group Analysis: Mean YTD Return for Extreme Utilization Groups

The chart below compares the mean YTD returns for stocks in the top and bottom 50, and top and bottom 1% of average utilization. This analysis confirms the trend observed in the decile analysis, with the highest utilization groups showing significantly higher mean YTD returns compared to the lowest utilization groups.



## Summary:

### Data Analysis Key Findings

*   The analysis shows a general trend where higher stock utilization deciles tend to have higher mean Year-to-Date (YTD) returns.
*   There is a significant increase in mean YTD return for the highest utilization decile.
*   Comparing extreme groups, the highest utilization groups (Top 50 and Top 1%) exhibit significantly higher mean YTD returns compared to the lowest utilization groups (Bottom 50 and Bottom 1%).

### Insights or Next Steps

*   The positive correlation between utilization rate and YTD return suggests a potential "short squeeze" effect or that highly utilized stocks are experiencing strong positive momentum.
*   Further analysis could investigate the drivers behind the strong performance of the highest utilization deciles and extreme groups, such as news events, sector concentration, or market sentiment.


# Market Flash: Most Shorted Stocks

**Heavily Shorted Stocks Significantly Outperforming YTD**

Our analysis of the R3K Securities Lending data for the period after December 31, 2024, reveals a strong correlation between high average utilization rates (a proxy for shorting activity) and Year-to-Date (YTD) stock returns.

**Key Findings:**

**Decile Analysis:**

*   Stocks in the highest utilization decile (Decile 10) have shown a remarkable Mean YTD Return of 21.8%. This is significantly higher than the returns observed in lower utilization deciles.
*   Conversely, lower utilization deciles generally exhibit lower mean and median YTD returns. For example, Decile 1 (lowest utilization) has a Mean YTD Return of 3.7%.

## Decile Analysis: Mean YTD Return by Utilization Decile

<!-- The chart for Mean YTD Return by Utilization Decile will be inserted here -->

**Extreme Group Analysis:**

*   The Top 50 Most Shorted stocks (based on average utilization) have experienced an impressive Mean YTD Return of 27.1% and a Median YTD Return of 2.5%.
*    In stark contrast, the Bottom 50 Least Shorted stocks have only yielded a Mean YTD Return of 5.3% and a Median YTD Return of 2.0%.
*   Looking at the most extreme cases, the Top 1% Most Shorted stocks (25 stocks) still show a strong Mean YTD Return of 11.9%. The Bottom 1% Least Shorted stocks (25 stocks) have a Mean YTD Return of 3.9%.

## Extreme Group Analysis: Mean YTD Return for Extreme Utilization Groups

<!-- The chart for Mean YTD Return for Extreme Utilization Groups will be inserted here -->

**Interpretation:**

The substantial outperformance of stocks with high average utilization rates, particularly in the top decile and the top 50 most shorted, strongly suggests that short sellers in these stocks have been under pressure. This pressure could be due to a variety of factors, including unexpected positive news, strong market momentum, or coordinated buying activity, forcing short sellers to buy back shares to cover their positions, thus driving prices up further.

The difference in returns between the most and least shorted groups is significant and warrants attention from market participants. This pattern is consistent with a short squeeze scenario, where high short interest fuels upward price momentum as short sellers are forced to cover their positions.

## Market Flash: Most Shorted Stocks

**Heavily Shorted Stocks Significantly Outperforming YTD**

Our analysis of the R3K Securities Lending data for the period after December 31, 2024, reveals a strong correlation between high average utilization rates (a proxy for shorting activity) and Year-to-Date (YTD) stock returns.

**Key Findings:**

**Decile Analysis:**

*   Stocks in the highest utilization decile (Decile 10) have shown a remarkable Mean YTD Return of 21.8%. This is significantly higher than the returns observed in lower utilization deciles.
*   Conversely, lower utilization deciles generally exhibit lower mean and median YTD returns. For example, Decile 1 (lowest utilization) has a Mean YTD Return of 3.7%.

## Decile Analysis: Mean YTD Return by Utilization Decile

{cell_id:51936665}

**Extreme Group Analysis:**

*   The Top 50 Most Shorted stocks (based on average utilization) have experienced an impressive Mean YTD Return of 27.1% and a Median YTD Return of 2.5%.
*    In stark contrast, the Bottom 50 Least Shorted stocks have only yielded a Mean YTD Return of 5.3% and a Median YTD Return of 2.0%.
*   Looking at the most extreme cases, the Top 1% Most Shorted stocks (25 stocks) still show a strong Mean YTD Return of 11.9%. The Bottom 1% Least Shorted stocks (25 stocks) have a Mean YTD Return of 3.9%.

## Extreme Group Analysis: Mean YTD Return for Extreme Utilization Groups

{cell_id:d837ba7f}

**Interpretation:**

The substantial outperformance of stocks with high average utilization rates, particularly in the top decile and the top 50 most shorted, strongly suggests that short sellers in these stocks have been under pressure. This pressure could be due to a variety of factors, including unexpected positive news, strong market momentum, or coordinated buying activity, forcing short sellers to buy back shares to cover their positions, thus driving prices up further.

The difference in returns between the most and least shorted groups is significant and warrants attention from market participants. This pattern is consistent with a short squeeze scenario, where high short interest fuels upward price momentum as short sellers are forced to cover their positions.

In [ ]:
import pandas as pd
import numpy as np

# --- 1. Load Data & Initial Setup ---
# We begin by loading the dataset. The 'business_date' column is parsed as a date object,
# which is crucial for time-series analysis. This ensures that we can correctly identify
# the start and end of the period for calculations like Year-to-Date returns.
try:
    df = pd.read_csv('R3K_SecLending_after_20241231_clean.csv', parse_dates=['business_date'])
    print("Step 1: Data loaded successfully.")
except FileNotFoundError:
    print("Error: 'R3K_SecLending_after_20241231_clean.csv' not found.")
    print("Please ensure the CSV file is in the same directory as this notebook.")
    # Exit or create a dummy dataframe to avoid further errors in the script
    df = pd.DataFrame()

if not df.empty:
    # --- 2. Calculate Value-Based Utilization ---
    # Value-Based Utilization is a key metric in securities lending that indicates how much of
    # a security's available inventory is being borrowed. It's calculated as the ratio of
    # the value of borrowed shares to the total inventory value.
    # We convert 'borrow_value' and 'inventory_val_amt' to numeric types, coercing any
    # errors into 'Not a Number' (NaN). This prevents non-numeric data from breaking our calculations.
    # Rows with missing essential data for this calculation are dropped to ensure data quality.
    # To handle cases where inventory value is zero (which would cause a division by zero error),
    # we replace the resulting 'infinity' values with NaN.
    df['borrow_value'] = pd.to_numeric(df['borrow_value'], errors='coerce')
    df['inventory_val_amt'] = pd.to_numeric(df['inventory_val_amt'], errors='coerce')
    df.dropna(subset=['borrow_value', 'inventory_val_amt'], inplace=True)
    df['utilization'] = df['borrow_value'] / df['inventory_val_amt']
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    print("Step 2: Value-Based Utilization calculated.")

    # --- 3. Apply Value-Based Gating ---
    # To focus our analysis on securities with significant lending activity, we apply a 'gating'
    # procedure. This filters the universe of stocks to include only those that meet certain
    # criteria on the most recent trading day in the dataset. This ensures our analysis is
    # focused on relevant, actively lent names.
    latest_date = df['business_date'].max()
    latest_day_data = df[df['business_date'] == latest_date]

    gating_criteria = (latest_day_data['borrow_value'] >= 1_000_000) & \
                      (latest_day_data['inventory_val_amt'] >= 10_000_000)

    gated_tickers = latest_day_data[gating_criteria]['ticker'].unique()
    gated_df = df[df['ticker'].isin(gated_tickers)].copy()
    print(f"Step 3: Gating applied. {len(gated_tickers)} stocks met the criteria.")

    # --- 4. Calculate YTD Returns & Average Utilization ---
    # For the gated stocks, we now calculate Year-to-Date (YTD) returns and average utilization.
    # YTD return is calculated as the percentage change in price from the first to the last
    # day of the period for each stock. Average utilization is the mean of the daily
    # utilization over the entire period. These two metrics will be the basis of our analysis.
    gated_df.sort_values(by=['ticker', 'business_date'], inplace=True)

    # Calculate YTD returns
    start_prices = gated_df.groupby('ticker')['security_price'].first()
    end_prices = gated_df.groupby('ticker')['security_price'].last()
    ytd_returns = ((end_prices - start_prices) / start_prices)
    ytd_returns.name = 'ytd_return'

    # Calculate average utilization
    avg_utilization = gated_df.groupby('ticker')['utilization'].mean()
    avg_utilization.name = 'avg_utilization'

    # Combine into a single analysis DataFrame
    analysis_df = pd.concat([ytd_returns, avg_utilization], axis=1)
    analysis_df.dropna(inplace=True) # Drop stocks with missing data for these metrics
    print("Step 4: YTD Returns and Average Utilization calculated.")
    print("\nSample of the analysis DataFrame:")
    print(analysis_df.head())


    # --- 5. Decile Analysis ---
    # To understand the relationship between utilization and performance, we perform a decile analysis.
    # Stocks are sorted by their average utilization and then grouped into ten equal-sized buckets (deciles).
    # We then calculate the mean YTD return for each decile. This may indicate if
    # higher utilization is associated with different return profiles.
    analysis_df['decile'] = pd.qcut(analysis_df['avg_utilization'], 10, labels=False, duplicates='drop')
    decile_analysis = analysis_df.groupby('decile')['ytd_return'].agg(['mean'])
    print("\n--- Step 5: Decile Analysis ---")
    print("Mean YTD Return by Utilization Decile:")
    print(decile_analysis)


    # --- 6. Extreme Group Analysis ---
    # This analysis focuses on the extremes. We look at the performance of the top and bottom 50
    # stocks by average utilization, as well as the top and bottom 1%. This can highlight
    # performance characteristics of the most and least crowded shorts in the market.
    # This may provide signals about market sentiment at its extremes.
    print("\n--- Step 6: Extreme Group Analysis ---")
    sorted_analysis_df = analysis_df.sort_values(by='avg_utilization', ascending=False)

    # Top and Bottom 50 Analysis
    top_50 = sorted_analysis_df.head(50)
    bottom_50 = sorted_analysis_df.tail(50)

    print("\nPerformance of Top 50 Stocks by Average Utilization:")
    print(top_50['ytd_return'].agg(['mean']))

    print("\nPerformance of Bottom 50 Stocks by Average Utilization:")
    print(bottom_50['ytd_return'].agg(['mean']))


    # Top and Bottom 1% Analysis
    # This is performed only if the gated universe is large enough to yield meaningful results.
    num_stocks = len(analysis_df)
    if num_stocks >= 100:
        one_percent_count = int(num_stocks * 0.01)
        if one_percent_count > 0:
            top_1_percent = sorted_analysis_df.head(one_percent_count)
            bottom_1_percent = sorted_analysis_df.tail(one_percent_count)

            print(f"\nPerformance of Top 1% ({one_percent_count} stocks) by Average Utilization:")
            print(top_1_percent['ytd_return'].agg(['mean']))

            print(f"\nPerformance of Bottom 1% ({one_percent_count} stocks) by Average Utilization:")
            print(bottom_1_percent['ytd_return'].agg(['mean']))
        else:
            print("\nNot enough stocks for a meaningful 1% analysis (1% is less than 1 stock).")
    else:
        print("\nSkipping 1% analysis because there are fewer than 100 stocks in the gated universe.")

### Top 50 Stocks by Average Utilization:

In [ ]:
display(top_50['ytd_return'])

### Bottom 50 Stocks by Average Utilization:

In [ ]:
display(bottom_50['ytd_return'])

### Top 1% Stocks by Average Utilization:

In [ ]:
display(top_1_percent['ytd_return'])

### Bottom 1% Stocks by Average Utilization:

In [ ]:
display(bottom_1_percent['ytd_return'])

# Methodology Document

This document outlines the methodology used to analyze the relationship between stock utilization rates and Year-to-Date (YTD) stock returns based on the provided R3K Securities Lending data.

## 1. Data Loading and Initial Setup

*   The dataset is loaded from the CSV file 'R3K\_SecLending\_after\_20241231\_clean.csv'.
*   The 'business\_date' column is parsed as a date object to enable time-series analysis.
*   Error handling is included to check for the presence of the data file.

## 2. Calculation of Value-Based Utilization

*   Value-Based Utilization is calculated as the ratio of 'borrow\_value' to 'inventory\_val\_amt'.
*   Both 'borrow\_value' and 'inventory\_val\_amt' are converted to numeric types, with errors coerced to NaN.
*   Rows with missing values in 'borrow\_value' or 'inventory\_val\_amt' are removed.
*   Infinite values resulting from division by zero are replaced with NaN.

## 3. Application of Value-Based Gating

*   The analysis is focused on stocks that meet specific criteria on the most recent trading day in the dataset.
*   The gating criteria are: 'borrow\_value' >= 1,000,000 and 'inventory\_val\_amt' >= 10,000,000.
*   A list of tickers that meet these criteria is generated.
*   The original DataFrame is filtered to include only the data for these gated tickers.

## 4. Calculation of YTD Returns and Average Utilization

*   The gated data is sorted by ticker and business date.
*   YTD return for each stock is calculated as the percentage change in 'security\_price' from the first to the last day of the period.
*   Average utilization for each stock is calculated as the mean of the 'utilization' values over the period.
*   YTD returns and average utilization are combined into a single analysis DataFrame.
*   Stocks with missing data for either metric in the analysis DataFrame are removed.

## 5. Decile Analysis

*   Stocks in the analysis DataFrame are sorted by their average utilization.
*   The sorted stocks are divided into ten equal-sized deciles based on average utilization.
*   The mean YTD return is calculated for each utilization decile.

## 6. Extreme Group Analysis

*   The analysis DataFrame is sorted by average utilization in descending order.
*   The top 50 and bottom 50 stocks by average utilization are identified.
*   The top 1% and bottom 1% of stocks by average utilization are identified, provided there are at least 100 stocks in the analysis universe.
*   The mean YTD return is calculated for each of these extreme groups (Top 50, Bottom 50, Top 1%, Bottom 1%).

In [ ]:
# Read the tickers from the Excel sheet
holdings_df = pd.read_excel("/content/R3K_Holdings.xlsx")
excel_tickers = set(holdings_df['Ticker'].unique())

# Get the tickers from the original CSV file
# Assuming the original dataframe before gating is 'df'
if not df.empty:
    csv_tickers = set(df['ticker'].unique())

    # Find the overlap
    overlap_tickers = excel_tickers.intersection(csv_tickers)

    print(f"Number of tickers from the Excel sheet present in the original CSV file: {len(overlap_tickers)}")
else:
    print("The original dataframe (df) is empty. Cannot determine overlap.")